<h1>Data Exploration Section<h1>

<h2>This section allows you to select and display video data from the input_videos folder

<h3>Install necessary libraries<h3>

In [1]:
!pip install opencv-python-headless
import os
import cv2
from tqdm import tqdm
from ipywidgets import interact, widgets
from IPython.display import display, HTML
from base64 import b64encode
from moviepy.editor import VideoFileClip

<h3>Clone github repo<h3>

In [2]:
print('Git clone project')
!git clone -b data-exploration https://github.com/YUyou0/ict3104-team14-2023.git

Git clone project
Cloning into 'ict3104-team14-2023'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 25 (delta 2), reused 20 (delta 1), pack-reused 0
Receiving objects: 100% (25/25), 27.45 MiB | 34.24 MiB/s, done.
Resolving deltas: 100% (2/2), done.


<h3>Define the Video Directory<h3>

In [3]:
video_dir = '/content/ict3104-team14-2023/input_videos/charades'

<h3>Upload Video Files into the Directory<h3>

In [ ]:
from google.colab import files
import shutil

# Upload a video file
uploaded = files.upload()

file_name = list(uploaded.keys())[0]  # Extract the uploaded file name
destination_path = video_dir     # Folder to store uploaded files

# Move the file
shutil.move(os.path.join(os.getcwd(), file_name), os.path.join(destination_path))

# Optionally, check if the file has been moved successfully
if not os.path.exists(os.path.join(os.getcwd(), file_name)):
    print("File has been successfully moved.")
else:
    print("File move operation failed.")

<h3>List Video Files in the Directory<h3>

In [4]:
video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi', '.mkv'))]

In [5]:
# Add empty as the first item to represent no selection initially
if video_files[0] != "":
    video_files.insert(0, "")

# Create a dropdown widget to select an item
item_dropdown = widgets.Dropdown(options=video_files, description=" ")  # hardcoded spacing

# Define a function that will run based on the selected item
def run_function(selected_item):
    if selected_item != "":
      print(" ") # hardcoded spacing

      # Construct the path to the selected video
      video_path = os.path.join(video_dir, selected_item)

      # Display the selected video using the display_video function
      display(display_video(video_path, width=1080, height=640, autoplay=True, loop=True, controls=True))

# Configure video display widget settings
def display_video(video_path, width=640, height=360, autoplay=True, loop=True, controls=True):
    video_file = open(video_path, "rb").read()
    video_url = "data:video/mp4;base64," + b64encode(video_file).decode()
    video_tag = f'<video width="{width}" height="{height}" autoplay={"autoplay" if autoplay else ""} loop={"loop" if loop else ""} {"controls" if controls else ""}><source src="{video_url}" type="video/mp4"></video>'
    return HTML(data=video_tag)

In [6]:
# function to get video duration
def get_video_duration(video_path):
    try:
        video = VideoFileClip(video_path)
        duration = video.duration
        return duration
    except Exception as e:
        print(f"Error: {e}")
        return None

# Iterate through video files to collect information
def collect_information():
    for video_file in video_files:
        if video_file != "":
            video_path = os.path.join(video_dir, video_file)
            file_size = os.path.getsize(video_path)
            duration = get_video_duration(video_path)
            if duration is not None:
                print(f"Filename: {video_file}\t Size (bytes): {file_size} bytes\t Duration (seconds): {duration} seconds")
            else:
                print(f"Unable to determine video duration for {video_file}")

<h3>Select a video from the list and display video output<h3>

In [7]:
# get basic information to display as list
collect_information()
print("Select a video to play: ")

# Create an interactive widget that triggers the function when an item is selected
item_dropdown.value = "" # hardcoded default setting
interactive_widget = widgets.interactive(run_function, selected_item=item_dropdown)

# Display the interactive widget
display(interactive_widget)

Filename: 4GLAP.mp4	 Size (bytes): 2843331 bytes	 Duration (seconds): 30.51 seconds
Filename: 5O0YS.mp4	 Size (bytes): 2735755 bytes	 Duration (seconds): 20.81 seconds
Filename: 0RJKT.mp4	 Size (bytes): 1863374 bytes	 Duration (seconds): 27.15 seconds
Filename: 52CKM.mp4	 Size (bytes): 2198189 bytes	 Duration (seconds): 33.9 seconds
Filename: 3IPI3.mp4	 Size (bytes): 1510774 bytes	 Duration (seconds): 18.0 seconds
Filename: 5EFQ7.mp4	 Size (bytes): 2654524 bytes	 Duration (seconds): 31.7 seconds
Filename: 3LMTS.mp4	 Size (bytes): 5105105 bytes	 Duration (seconds): 30.35 seconds
Filename: 5D3X6.mp4	 Size (bytes): 1997096 bytes	 Duration (seconds): 17.28 seconds
Filename: 2E8GU.mp4	 Size (bytes): 2928833 bytes	 Duration (seconds): 22.43 seconds
Filename: 8ASRQ.mp4	 Size (bytes): 1961188 bytes	 Duration (seconds): 18.21 seconds
Filename: 3JJ7C.mp4	 Size (bytes): 3196414 bytes	 Duration (seconds): 31.74 seconds
Select a video to play: 


interactive(children=(Dropdown(description=' ', options=('', '4GLAP.mp4', '5O0YS.mp4', '0RJKT.mp4', '52CKM.mp4…